In [23]:
from shapely.geometry import Polygon
import pandas as pd
import json
import numpy as np
from ast import literal_eval

In [2]:
bh_df = pd.read_csv('bh-annotation.csv')

# Deal with the bh-annotation data

In [3]:
# Used to store the features of holds
shape_features = []
centers = []

for idx, row in bh_df.iterrows():
    try:
        shape_info = json.loads(row['region_shape_attributes'])
        # If the comment is not polygon (such as Ellipses and rectangles), then records NaN
        if shape_info.get('name') != 'polygon':
            shape_features.append((np.nan, np.nan, np.nan, np.nan))
            centers.append((np.nan, np.nan))
            continue
        
        all_x = shape_info['all_points_x']
        all_y = shape_info['all_points_y']
        # If node is less than 3, then it cant construct polygon 
        if len(all_x) < 3 or len(all_y) < 3:
            shape_features.append((np.nan, np.nan, np.nan, np.nan))
            centers.append((np.nan, np.nan))
            continue

        # Combine x and y, e.g. [(x1,y1),(x2,y2), ...]
        points = list(zip(all_x, all_y))
        # Use it to create a 多边形对象
        poly = Polygon(points)
        
        area = poly.area
        perimeter = poly.length
        aspect_ratio = (poly.bounds[2] - poly.bounds[0]) / (poly.bounds[3] - poly.bounds[1]) if (poly.bounds[3] - poly.bounds[1]) != 0 else 0
        circularity = 4 * np.pi * area / (perimeter ** 2) if perimeter != 0 else 0
        
        shape_features.append((area, perimeter, aspect_ratio, circularity))
        center_x = np.mean(all_x)
        center_y = np.mean(all_y)
        centers.append((center_x, center_y))
    except Exception as e:
        shape_features.append((np.nan, np.nan, np.nan, np.nan))
        centers.append((np.nan, np.nan))

In [4]:
bh_df['center_x'] = [c[0] for c in centers]
bh_df['center_y'] = [c[1] for c in centers]
bh_df['shape_area'] = [s[0] for s in shape_features]
bh_df['shape_perimeter'] = [s[1] for s in shape_features]
bh_df['shape_aspect_ratio'] = [s[2] for s in shape_features]
bh_df['shape_circularity'] = [s[3] for s in shape_features]

In [5]:
df_cleaned = bh_df.dropna(subset=['center_x','center_y'])

In [7]:
df_cleaned = df_cleaned.drop(
    columns=["file_size",
             "file_attributes",
             "region_count",
             "region_id",
             "region_shape_attributes",
             "region_attributes"]
)

In [ ]:
# Contain the center location of holds, and the attributes of holds
df_cleaned

,filename,center_x,center_y,shape_area,shape_perimeter,shape_aspect_ratio,shape_circularity
0,0000.jpg,898.200000,1115.866667,8463.0,350.548866,0.790698,0.865440
1,0000.jpg,2064.250000,200.500000,35619.5,698.205728,0.836066,0.918186
2,0000.jpg,2383.750000,1556.250000,5830.0,296.728826,1.207317,0.832068
3,0000.jpg,919.846154,1739.923077,2008.0,166.220468,0.827586,0.913281
4,0000.jpg,713.807692,369.730769,15616.0,561.903101,1.321918,0.621522
...,...,...,...,...,...,...,...
2505,1037.jpg,2487.421053,230.526316,5617.5,304.220222,1.120879,0.762740
2506,1037.jpg,2501.238095,44.285714,5185.0,279.747596,1.197531,0.832580
2507,1037.jpg,1799.444444,482.333333,9680.5,358.154237,0.744186,0.948347
2508,1037.jpg,2675.636364,142.909091,42090.0,822.913427,0.820000,0.781053


In [9]:
df_cleaned.to_csv("climbing_with_shape_features.csv",index=False)